In [6]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

import time
from tqdm import tqdm

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    # Split the population into group A (1/4) and group B (3/4)
    group_A_size = n // 4
    group_B_size = n - group_A_size
    
    # Generate x with a bias for group A
    x = np.zeros((n, m, p))
    x[:group_A_size] = rnd.normal(0.5, 1, (group_A_size, m, p))  # Slightly higher mean for group A
    x[group_A_size:] = rnd.normal(0, 1, (group_B_size, m, p))   # Standard distribution for group B
    
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            
            # Introduce bias for c: Group A has slightly higher values, Group B slightly lower
            if i < group_A_size:
                values *= 1.1  # Increase c for Group A
            else:
                values *= 0.9  # Decrease c for Group B
            
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            # Generate r using normal distribution, then clip to [0, 1] range
            r[i, j] = np.clip(rnd.normal(0.5, 0.2), 0, 1)
            
            # Introduce bias for r: Group A has slightly lower values, Group B slightly higher
            if i < group_A_size:
                r[i, j] -= 0.1  # Decrease r for Group A
            else:
                r[i, j] += 0.1  # Increase r for Group B
            
            # Clip again to ensure values remain in [0, 1] range
            r[i, j] = np.clip(r[i, j], 0, 1)

    return x, r, c, Q

In [ ]:
class optModelClosed:
    """
    This is a class for optimization models.
    """

    def __init__(self, r, c, Q, alpha=0.5):
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data = num_data
        self.num_items = num_items
        
    def __repr__(self):
        return 'optModel ' + self.__class__.__name__
    
    def setObj2(self, a, r, b, c, Q, epsilon=0.01, alpha=0.5):
        if alpha == 1:
            self.objective = cp.sum(cp.log(a * r + b * self.d + epsilon))
        else:
            self.objective = cp.sum(cp.power(a * r + b * self.d + epsilon, 1 - alpha)) / (1 - alpha)
        
        self.constraints = [
            cp.sum(cp.multiply(c, self.d)) <= Q
        ]
        
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)
        self.a = a
        self.r = r
        self.b = b
        self.c = c
        self.Q = Q
        self.epsilon = epsilon


    def setObj(self, r, c):

        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)

    
    def solveS(self, closed=False):
        """
        A method to solve the optimization problem for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization
            closed (bool): solving the problem in closed form

        Returns:
            tuple: optimal solution and optimal value
        """
        if closed:
            return self.solveC()

        self.d = cp.Variable(self.num_items)
        self.setObj(self.r, self.c)
        self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
        opt_sol = self.d.value
        opt_val = self.problem.value

        return opt_sol, opt_val


    def solve(self):
        """
        A method to solve the optimization problem in closed form for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization

        Returns:
            tuple: optimal solution and optimal value
        """
        r = self.r
        c = self.c
        if self.alpha == 1:
            raise ValueError("Work in progress")
        c = c.cpu().numpy() if isinstance(c, torch.Tensor) else c
        r = r.cpu().numpy() if isinstance(r, torch.Tensor) else r
        S = np.sum(c ** (1 - 1 / self.alpha) * r ** (-1 + 1 / self.alpha))
        opt_sol_c = (c ** (-1 / self.alpha) * r ** (-1 + 1 / self.alpha) * self.Q) / S
        opt_val_c = np.sum((r * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)

        return opt_sol_c, opt_val_c
    
    def solveC2(self):
        """
        A method to solve the optimization problem in closed form using the given formula.

        Returns:
            tuple: optimal solution and optimal value
        """
        if self.alpha == 1:
            raise ValueError("Work in progress")

        a = self.a
        r = self.r
        b = self.b
        c = self.c
        epsilon = self.epsilon
        Q = self.Q
        alpha = self.alpha

        b_inverse_alpha = b ** (-1 / alpha)
        c_over_b = c / b
        ar_plus_epsilon = a * r + epsilon

        S1 = np.sum(c_over_b * ar_plus_epsilon)
        S2 = np.sum(c_over_b ** (1 - 1 / alpha))

        d_star = b_inverse_alpha * (Q + S1) / S2 - ar_plus_epsilon * b_inverse_alpha / b

        opt_val_c2 = np.sum((a * r + b * d_star + epsilon) ** (1 - alpha)) / (1 - alpha)

        return d_star, opt_val_c2